In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, Markdown
from collections import Counter
import random
from lightgbm import LGBMClassifier

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import lightning as L
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchmetrics.classification import AUROC
from lightning.pytorch.callbacks import EarlyStopping

/net/tscratch/people/plgmatpat/miniconda3/envs/hallu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EXP_NAME = 'gemma_att'

In [4]:
os.getcwd()

'/net/tscratch/people/plgmatpat'

In [5]:
context_df = None

for f_ in os.listdir(os.path.join(EXP_NAME)):
    
    if f_.startswith('attension'):

        if context_df is None:
            context_df = pd.read_parquet(os.path.join(EXP_NAME, f_))
        else:
            context_df = pd.concat((context_df, pd.read_parquet(os.path.join(EXP_NAME, f_))))

In [6]:
context_df['dataset'].value_counts()

dataset
cnndm          19400
nq              8030
xsum            7776
poquad_v2       6247
hotpotqa_en     3121
bioask          3056
hotpotqa_pl     2208
polqa           1869
Name: count, dtype: int64

In [7]:
context_df = context_df[context_df['dataset'].isin(['xsum', 'nq'])]

In [8]:
context_df['label'].value_counts()

label
0    14012
1     1794
Name: count, dtype: int64

In [9]:
import torch
from torch.utils.data import Dataset

class HalluDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim

    def forward(self, x):

        if len(x.shape) == 2:
            x = x.unsqueeze(0)

        _, (hn, _) = self.lstm(x)  # hn is (num_layers, batch_size, hidden_dim)
        out = self.fc(hn[-1])  # Use the last layer's hidden state
        return out

In [11]:
class LSTMClassifier(L.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, lr=5e-4, num_layers=1, cls_weight=None):
        super().__init__()
        self.model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers=num_layers)
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=cls_weight)
        self.lr = lr

        self.test_dataloader = None

        # AUROC metric for binary classification
        self.train_auc = AUROC(task="binary")
        self.val_auc = AUROC(task="binary")
        self.test_auc = AUROC(task="binary")
        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        preds = self(data)
        labels = labels.view(-1, 1).float() 

        if preds.size() != labels.size():
            preds = preds.view_as(labels)

        loss = self.criterion(preds, labels)
        # Calculate AUC during training
        prob = torch.sigmoid(preds)
        auc = self.train_auc(prob, labels)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_auc", auc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        preds = self(data)
        labels = labels.view(-1, 1).float() 


        loss = self.criterion(preds, labels)
        # Calculate AUC during validation
        prob = torch.sigmoid(preds)
        auc = self.val_auc(prob, labels)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_auc", auc, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        data, labels = batch
        preds = self(data)
        labels = labels.view(-1, 1).float() 
        loss = self.criterion(preds, labels)
        # Calculate AUC during validation
        prob = torch.sigmoid(preds)
        auc = self.val_auc(prob, labels)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_auc", auc, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        for i, batch in enumerate(self.test_dataloader):
            batch = self.transfer_batch_to_device(batch, self.device, 0)
            self.test_step(batch, i)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [12]:
train_cols = [col for col in context_df.columns if col not in ['dataset', 'label']]

In [13]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from collections import Counter
from pytorch_lightning.loggers import TensorBoardLogger

In [14]:
SPLIT_VAL = True
INPUT_DIM = 16
GEMMA_LAYERS = 42
HIDDEN_DIM = 64
OUTPUT_DIM = 1
NUM_LSTM_LAYERS = 2
LR = 1e-3
MAX_EPOCHS = 25
BATCH_SIZE = 1024

In [15]:
from lightning.pytorch.loggers import WandbLogger 
import wandb

In [16]:
validation_results = []

# Loop over each unique dataset
for dataset in context_df['dataset'].unique():

    # run = wandb.init(entity=dataset, project="hallu_project")

    in_dist_sample = context_df.loc[context_df['dataset'] != dataset]
    out_dist_sample = context_df.loc[context_df['dataset'] == dataset]

    X_train, X_test = in_dist_sample[train_cols], out_dist_sample[train_cols]
    y_train, y_test = in_dist_sample['label'].to_numpy().astype(np.int64), out_dist_sample['label'].to_numpy().astype(np.int64)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

    counter = Counter(y_train)
    pos_weight = torch.tensor([counter[0] / counter[1]], dtype=torch.float32)

    # rus = RandomUnderSampler(random_state=42)
    # X_train, y_train = rus.fit_resample(X_train, y_train)

    scaler = RobustScaler()

    X_train = scaler.fit_transform(X_train).reshape(-1, GEMMA_LAYERS, INPUT_DIM)
    X_val = scaler.transform(X_val).reshape(-1, GEMMA_LAYERS, INPUT_DIM)
    X_test = scaler.transform(X_test).reshape(-1, GEMMA_LAYERS, INPUT_DIM)

    train_dataset = HalluDataset(X_train, y_train)
    val_dataset = HalluDataset(X_val, y_val)
    test_dataset = HalluDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=16)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=16)


    model = LSTMClassifier(
        input_dim=INPUT_DIM, 
        hidden_dim=HIDDEN_DIM, 
        output_dim=OUTPUT_DIM, 
        lr=LR, 
        num_layers=NUM_LSTM_LAYERS,
        cls_weight=pos_weight
    )

    model.test_dataloader = test_loader

    wandb_logger = WandbLogger(
        # name=f"lstm_on_{dataset}",
        project="hallu_project",
        group=f"lstm_hallu",
        reinit=True
    )

    wandb_logger.experiment.config.update(
        {
            "trained_on": set(context_df['dataset'].unique()) - {dataset},
            "tested_on": dataset,
        }
    )

    # wandb_logger.log()

    trainer = L.Trainer(
        max_epochs=MAX_EPOCHS,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        log_every_n_steps=10,
        callbacks=[EarlyStopping(
            monitor="val_loss",  # The metric to monitor
            patience=7,          # Number of epochs with no improvement after which training will stop
            verbose=True,        # Display a message when stopping
            mode="min",          # Minimize the monitored metric (for loss)
        )],
        logger=wandb_logger
    )

    trainer.fit(model, train_loader, val_loader)

    metrics = trainer.callback_metrics
    res = {k: v.item() for k, v in metrics.items()}

    test_res = trainer.test(model, test_loader)[0]
    print(f"Test Loss: {test_res['test_loss']}, Test AUC: {test_res['test_auc']}")

    validation_results.append({
        'dataset': dataset,
        'train_loss': res['train_loss'],
        'val_loss': res['val_loss'],
        'test_loss': res['test_loss'],
        'train_auc': res['train_auc'],
        'val_auc': res['val_auc'],
        'test_auc': res['test_auc']
    })

    wandb_logger.finalize("success")
    wandb.finish()

/net/tscratch/people/plgmatpat/miniconda3/envs/hallu/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: matpat (hallucination). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | LSTMModel         | 54.3 K | train
1 | criterion | BCEWithLogitsLoss | 0      | train
2 | train_auc | BinaryAUROC       | 0      | train
3 | val_auc   | BinaryAUROC       | 0      | train
4 | test_auc  | BinaryAUROC       | 0      | train
--------------------------------------------------------
54.3 K    Trainable params
0         Non-trainable params
5

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/net/tscratch/people/plgmatpat/miniconda3/envs/hallu/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


/net/tscratch/people/plgmatpat/miniconda3/envs/hallu/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Epoch 24: 100%|██████████| 7/7 [00:04<00:00,  1.55it/s, v_num=1qqe, val_loss=1.020, val_auc=0.767, train_loss=1.030, train_auc=0.757, test_loss=1.220, test_auc=0.533]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


SLURM auto-requeueing enabled. Setting signal handlers.


Testing DataLoader 0: 100%|██████████| 8/8 [00:00<00:00, 73.39it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_auc            0.5456796884536743
        test_loss            1.154437780380249
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Loss: 1.154437780380249, Test AUC: 0.5456796884536743


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
test_auc,▁▁▁▂▂▃▃▄▅▇▆▄▇█▇▇████▇▇▇▆▇▇
test_loss,▂██▄▃▂▃▂▁▂▆█▄▃▂▃▃▄▅▄▂▁▁▅▁▁
train_auc,▁▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train_loss,█▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
val_auc,▁▁▂▂▂▃▃▄▄▅▅▇▇▇▇▇▇▇█████▇█
val_loss,█▇▇▇▆▆▆▆▅▅▄▄▄▃▃▃▃▂▂▃▂▁▂▂▁
epoch,25
test_auc,0.54568
test_loss,1.15444
